![title](img/ESPN.png)

In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm

LastGames = 10
print(f"LastGames: {LastGames}")
score_ranges = list(range(0, 7))

History = pd.read_csv('History.csv')

# 1. Clean and Sort
History.drop_duplicates(inplace=True)
# Sorting by Date is crucial to ensure iloc[:LastGames] gets the correct games
if 'Date' in History.columns:
    History['Date'] = pd.to_datetime(History['Date'])
    History = History.sort_values('Date', ascending=False)

# 2. Filter for teams with enough games
groups = History.groupby('Home')
h = pd.concat([g.iloc[:LastGames] for _, g in groups if len(g) >= LastGames])

# 3. Vectorized Column Creation
for r in tqdm(score_ranges, desc="Calculating Score Stats"):
    h[f'B>{r}'] = (h['But'] > r).astype(int)
    h[f'B<{r}'] = (h['But'] < r).astype(int)
    
    # Conditional stats based on DH, HA, DA flags
    h[f'DHB>{r}'] = ((h['But'] > r) & (h['DH'] == 1)).astype(int)
    h[f'DHB<{r}'] = ((h['But'] < r) & (h['DH'] == 1)).astype(int)
    
    h[f'HAB>{r}'] = ((h['But'] > r) & (h['HA'] == 1)).astype(int)
    h[f'HAB<{r}'] = ((h['But'] < r) & (h['HA'] == 1)).astype(int)
    
    h[f'DAB>{r}'] = ((h['But'] > r) & (h['DA'] == 1)).astype(int)
    h[f'DAB<{r}'] = ((h['But'] < r) & (h['DA'] == 1)).astype(int)

h['n'] = 1

# 4. Define Columns to Aggregate
# Generate the dynamic list of columns we just created
dynamic_cols = []
for r in score_ranges:
    dynamic_cols.extend([f'B>{r}', f'B<{r}', f'DHB>{r}', f'DHB<{r}', 
                         f'HAB>{r}', f'HAB<{r}', f'DAB>{r}', f'DAB<{r}'])

# The static columns already present in History
static_cols = [
    'Ds', 'Hs', 'As', 'DA', 'DH', 'HA', 'LDEMs', 'LDEMPs', 
    'DALDEMs', 'DHLDEMs', 'HALDEMs', 'DALDEMPs', 'DHLDEMPs', 'HALDEMPs', 'n'
]

all_cols = dynamic_cols + static_cols

# 5. Generate HomeStats
HomeStats = h.groupby('Home')[all_cols].sum()
HomeStats = HomeStats[HomeStats["n"] >= 6]
HomeStats = HomeStats.add_prefix('1').reset_index().rename(columns={'1Home': 'Home'})

# 6. Generate AwayStats
# Note: We use the same 'h' dataframe, but group by the 'Away' column
AwayStats = h.groupby('Away')[all_cols].sum()
AwayStats = AwayStats[AwayStats["n"] >= 6]
AwayStats = AwayStats.add_prefix('2').reset_index().rename(columns={'2Away': 'Away'})

# Display result
print(f"Home stats shape: {HomeStats.shape}")
print(f"Away stats shape: {AwayStats.shape}")

LastGames: 10


Calculating Score Stats: 100%|███████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 50.26it/s]


Home stats shape: (444, 72)
Away stats shape: (414, 72)


In [19]:
AwayStats

,Away,2B>0,2B<0,2DHB>0,2DHB<0,2HAB>0,2HAB<0,2DAB>0,2DAB<0,2B>1,...,2HA,2LDEMs,2LDEMPs,2DALDEMs,2DHLDEMs,2HALDEMs,2DALDEMPs,2DHLDEMPs,2HALDEMPs,2n
0,1. FC Heidenheim,9,0,5,0,6,0,7,0,8,...,6,5,4,4,4,2,3,1,4,9
1,1. FC Heidenheim 1846,8,0,7,0,7,0,2,0,8,...,7,7,1,2,6,6,0,1,1,8
2,1. FC Magdeburg,8,0,5,0,7,0,4,0,7,...,7,6,3,4,3,5,1,3,2,9
3,1. FC Union Berlin,9,0,5,0,7,0,6,0,5,...,7,4,5,2,4,2,4,1,5,9
4,AC Horsens,10,0,9,0,9,0,2,0,10,...,9,7,3,2,6,6,0,3,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Winterthur,8,0,6,0,6,0,4,0,7,...,6,6,2,3,5,4,1,1,2,8
410,Wolfsberger,9,0,4,0,9,0,5,0,8,...,9,4,6,1,3,4,5,2,5,10
411,Wolverhampton Wanderers,10,0,9,0,8,0,3,0,9,...,8,6,5,3,5,4,1,5,4,11
412,Young Boys,11,0,9,0,7,0,6,0,11,...,7,8,3,4,8,4,2,1,3,11


In [16]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# 1. Data Preparation
# Assuming 'History' is your original DataFrame
HistoryHA = pd.DataFrame(History)

# Create 'Team' column by stacking Home and Away
home_side = HistoryHA.copy()
home_side['Team'] = home_side['Home']

away_side = HistoryHA.copy()
away_side['Team'] = away_side['Away']

HistoryHA = pd.concat([home_side, away_side], ignore_index=True)

# Clean up Dates
HistoryHA["Date"] = pd.to_datetime(HistoryHA["Date"], format="%a, %d %b %Y", errors='coerce')
HistoryHA = HistoryHA.dropna(subset=['Team', 'Date', 'Score'])
HistoryHA = HistoryHA.sort_values(by=["Team", "Date"], ascending=False)

# 2. Robust Score Extraction
def extract_scores(score_str):
    if pd.isna(score_str):
        return pd.Series([0, 0])
    nums = re.findall(r'\d+', str(score_str))
    if len(nums) >= 2:
        return pd.Series([int(nums[0]), int(nums[1])])
    return pd.Series([0, 0])

HistoryHA[['ScoreH', 'ScoreA']] = HistoryHA['Score'].apply(extract_scores)
HistoryHA['But'] = HistoryHA['ScoreH'] + HistoryHA['ScoreA']

# 3. Determine Result Logic
def determine_result(row):
    if row['ScoreH'] == row['ScoreA']: 
        return 'D'
    # We compare against the 'Team' column we created during the stack
    is_home = row['Team'] == row['Home']
    if (row['ScoreH'] > row['ScoreA'] and is_home) or (row['ScoreH'] < row['ScoreA'] and not is_home):
        return 'W'
    return 'L'

HistoryHA['Result'] = HistoryHA.apply(determine_result, axis=1)

# 4. Optimized Streak Function (Fixed KeyError)
def calculate_streaks(group):
    # group.name is the 'Team' value for this specific group
    current_team_name = group.name 
    
    res_map = {
        "W": ["W", "WD", "WL"], 
        "D": ["D", "WD", "LD"], 
        "L": ["L", "LD", "WL"]
    }
    
    cols = ['W','L','D','WD','LD','WL','HW','HL','HD','HWD','HLD','HWL','AW','AL','AD','AWD','ALD','AWL']
    s = {k: 0 for k in cols}
    active = {k: True for k in cols}

    for _, row in group.iterrows():
        res = row['Result']
        current_res_types = res_map[res]
        
        # Use current_team_name instead of row['Team'] to avoid KeyError
        is_home = current_team_name == row['Home']
        loc_prefix = "H" if is_home else "A"

        # Check Overall Streaks
        for rt in ['W','L','D','WD','LD','WL']:
            if active[rt]:
                if rt in current_res_types: 
                    s[rt] += 1
                else: 
                    active[rt] = False
        
        # Check Location-Specific Streaks
        for rt in ['W','L','D','WD','LD','WL']:
            loc_key = loc_prefix + rt
            if active[loc_key]:
                if rt in current_res_types: 
                    s[loc_key] += 1
                else: 
                    active[loc_key] = False
        
        # Stop if all streaks are broken
        if not any(active.values()):
            break
            
    return pd.Series(s)

# 5. Apply and Aggregate
print("Calculating current streaks...")
# Using include_groups=False is the modern way to handle this in Pandas 2.0+
Streak = HistoryHA.groupby('Team').apply(calculate_streaks, include_groups=False)

# Calculate Maxima
Streak['max'] = Streak[['W','L','D','WD','LD','WL']].max(axis=1)
Streak['maxH'] = Streak[['HW','HL','HD','HWD','HLD','HWL']].max(axis=1)
Streak['maxA'] = Streak[['AW','AL','AD','AWD','ALD','AWL']].max(axis=1)

# Formatting
Streak = Streak.sort_values(['max', 'maxH'], ascending=False).reset_index()

# Display
print("Streaks calculated successfully.")
Streak.head(10)

Calculating current streaks...
Streaks calculated successfully.


,Team,W,L,D,WD,LD,WL,HW,HL,HD,...,HWL,AW,AL,AD,AWD,ALD,AWL,max,maxH,maxA
0,FC Salzburg,3,0,0,28,0,3,2,0,0,...,2,1,0,0,14,0,1,28,15,14
1,Barcelona,0,1,0,0,1,22,13,0,0,...,22,0,1,0,0,1,12,22,22,12
2,Lugo,0,2,0,0,20,2,0,1,0,...,1,0,4,0,0,11,4,20,10,11
3,Lens,3,0,0,3,0,18,11,0,0,...,18,1,0,0,1,0,9,18,18,9
4,Feyenoord Rotterdam,0,3,0,0,3,18,0,1,0,...,8,0,3,0,0,3,10,18,8,10
5,Sheffield United,1,0,0,1,0,17,5,0,0,...,15,1,0,0,1,0,8,17,15,8
6,Real Madrid,2,0,0,2,0,16,5,0,0,...,18,1,0,0,1,0,9,16,18,9
7,Gazisehir Gaziantep,0,12,0,0,12,16,0,7,0,...,10,0,5,0,0,5,6,16,10,6
8,Hatayspor,0,12,0,0,12,16,0,6,0,...,8,0,8,0,0,10,8,16,8,10
9,Lyon,12,0,0,12,0,15,8,0,0,...,20,5,0,0,5,0,7,15,20,7


In [18]:
# Newcastle United
HistoryHA[(HistoryHA["Team"] == "Lens")].head(10)

,Date,Home,Score,Away,Competition,ScoreH,ScoreA,But,Result,Ds,...,LDEMs,LDEMPs,DALDEMs,DHLDEMs,HALDEMs,DALDEMPs,DHLDEMPs,HALDEMPs,IdGame,Team
60,2026-02-07,Lens,3 - 1,Stade Rennais,fra.1,3,1,4,W,0,...,1,0,0,1,1,0,0,0,2026-02-07LensStade Rennais,Lens
10816,2026-02-04,Troyes,2 - 4,Lens,fra.2,2,4,6,W,0,...,1,0,1,0,1,0,0,0,2026-02-04TroyesLens,Lens
154,2026-01-30,Lens,1 - 0,Le Havre AC,fra.1,1,0,1,W,0,...,0,1,0,0,0,0,1,1,2026-01-30LensLe Havre AC,Lens
10959,2026-01-24,Marseille,3 - 1,Lens,fra.1,3,1,4,L,0,...,1,0,0,1,1,0,0,0,2026-01-24MarseilleLens,Lens
297,2026-01-17,Lens,1 - 0,AJ Auxerre,fra.1,1,0,1,W,0,...,0,1,0,0,0,0,1,1,2026-01-17LensAJ Auxerre,Lens
11094,2026-01-11,Sochaux,0 - 3,Lens,fra.1,0,3,3,W,0,...,0,1,0,0,0,1,0,1,2026-01-11SochauxLens,Lens
11204,2026-01-02,Toulouse,0 - 3,Lens,fra.1,0,3,3,W,0,...,0,1,0,0,0,1,0,1,2026-01-02ToulouseLens,Lens
562,2025-12-19,Lens,3 - 1,Feignies,fra.1,3,1,4,W,0,...,1,0,0,1,1,0,0,0,2025-12-19LensFeignies,Lens
600,2025-12-14,Lens,2 - 0,Nice,fra.1,2,0,2,W,0,...,0,1,0,0,0,0,1,1,2025-12-14LensNice,Lens
11446,2025-12-06,Nantes,1 - 2,Lens,fra.1,1,2,3,W,0,...,1,0,1,0,1,0,0,0,2025-12-06NantesLens,Lens
